# Challenge Scratchbook

* This notebook explores methods for the Kernel Methods for Machine Learning Kaggle [challenge](https://www.kaggle.com/c/kernel-methods-for-machine-learning-2018-2019/data).

* Note that this is a binary classification challenge.

Our first goal is to implement two baseline methods:
1. Random classification
2. All instances are 0s (Doing so we get an idea of the proportion of 0's in the public test set)
3. Implement the Simple Pattern Recognition Algorithm (SPR) from Learning with Kernels 

Before that, we have to implement some data loaders


Now that we are done with the above, our goal is to implement SVM with Gaussian kernel.

## Imports

In [1]:
import csv
import os
import numpy as np
from scipy import optimize
from tqdm import tqdm_notebook
import pandas as pd
from itertools import product
import scipy

from utils.data import load_data, save_results, cross_validation
from utils.models import SVM, SPR
from utils.kernels import GaussianKernel, LinearKernel

## Paths and Globals

In [2]:
CWD = os.getcwd()
DATA_DIR = os.path.join(CWD, "data")
RESULT_DIR = os.path.join(CWD, "results")

FILES = {0: {"train_mat": "Xtr0_mat100.csv",
             "train": "Xtr0.csv",
             "test_mat": "Xte0_mat100.csv",
             "test": "Xte0.csv",
             "label": "Ytr0.csv"},
         1: {"train_mat": "Xtr1_mat100.csv",
             "train": "Xtr1.csv",
             "test_mat": "Xte1_mat100.csv",
             "test": "Xte1.csv",
             "label": "Ytr1.csv"},
         2: {"train_mat": "Xtr2_mat100.csv",
             "train": "Xtr2.csv",
             "test_mat": "Xte2_mat100.csv",
             "test": "Xte2.csv",
             "label": "Ytr2.csv"}}

## 0 entries

In [3]:
if False:
    with open(os.path.join(RESULT_DIR, "results.csv"), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(["Id", "Bound"])
        for i in range(3000):
            writer.writerow([i, 0])

**Comment:**

* We get 0.51266 which means that the dataset is pretty balanced.

## SPR

* Simple Pattern Recognition algorithm with Gaussian kernel

In [5]:
γ = 500
λ = 5e-5
kernel = GaussianKernel(γ)
#kernel = LinearKernel()

len_files = len(FILES)
for i in range(len_files):
    X_train, Y_train, X_test = load_data(i, data_dir=DATA_DIR, files_dict=FILES)
    X_val = X_train[1600:]
    Y_val = Y_train[1600:]
    X_train = X_train[:1600]
    Y_train = Y_train[:1600]
    clf = SPR(kernel)
    clf.fit(X_train, Y_train)
    y_pred_train =clf.predict(X_train)
    y_pred_val = clf.predict(X_val)
    score_train = clf.score(y_pred_train, Y_train)
    score_val = clf.score(y_pred_val, Y_val)

    print(f"Accuracy on train set / val set {i} : {score_train} / {score_val} (λ: {λ},γ: {γ})")

Accuracy on train set / val set 0 : 0.99875 / 0.585 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 1 : 1.0 / 0.705 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 2 : 1.0 / 0.5775 (λ: 5e-05,γ: 500)


## Train and test on the different sets

In [4]:
results = np.zeros(3000)

for i in range(len(FILES)):
    X_train, Y_train, X_test = load_data(i, data_dir=DATA_DIR, files_dict=FILES)
    clf = SPR()
    clf.fit(X_train, Y_train)
    results[i*1000:i*1000 + 1000] = clf.predict(X_test)

### Save results

In [6]:
# Test the save results function
save_results("test_results.csv", results, result_dir=RESULT_DIR)

## SVM with Gaussian Kernel

### Comparison with ``scikit-learn`` implementation

In [10]:
γ = 500
λ = 5e-5
kernel = GaussianKernel(γ)

len_files = len(FILES)
for i in range(len_files):
    X_train, Y_train, X_test = load_data(i, data_dir=DATA_DIR, files_dict=FILES)
    X_val = X_train[1600:]
    Y_val = Y_train[1600:]
    X_train = X_train[:1600]
    Y_train = Y_train[:1600]
    clf = SVM(_lambda=λ, kernel=kernel)
    clf.fit(X_train, Y_train)
    y_pred_train =clf.predict(X_train)
    y_pred_val = clf.predict(X_val)
    score_train = clf.score(y_pred_train, Y_train)
    score_val = clf.score(y_pred_val, Y_val)

    print(f"Accuracy on train set / val set {i} : {score_train} / {score_val} (λ: {λ},γ: {γ})")

Accuracy on train set / val set 0 : 1.0 / 0.575 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 1 : 1.0 / 0.7275 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 2 : 1.0 / 0.6375 (λ: 5e-05,γ: 500)


In [11]:
n = 2000
print(f"C: {1/(2 * n * λ)}")

C: 5.0


In [13]:
from sklearn.svm import SVC

len_files = len(FILES)
for i in range(len_files):
    X_train, Y_train, X_test = load_data(i, data_dir=DATA_DIR, files_dict=FILES)
    X_val = X_train[1600:]
    Y_val = Y_train[1600:]
    X_train = X_train[:1600]
    Y_train = Y_train[:1600]
    clf = SVC(C=5.0, kernel="rbf", gamma=500)
    clf.fit(X_train, Y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_val = clf.predict(X_val)
    score_train = np.sum([Y_train == y_pred_train]) / len(Y_train)
    score_val = np.sum([Y_val == y_pred_val]) / len(Y_val)

    print(f"Accuracy on train set / val set {i} : {score_train} / {score_val} (λ: {λ},γ: {γ})")

Accuracy on train set / val set 0 : 1.0 / 0.5725 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 1 : 1.0 / 0.705 (λ: 5e-05,γ: 500)
Accuracy on train set / val set 2 : 1.0 / 0.5875 (λ: 5e-05,γ: 500)


### Investigate and improve Kmeans

* Diagnostic: introduce a tolerance parameter ``tol``

### Cross-validation

In [3]:
def cross_validation(dataset_idx, clf, k=5, embeddings=None):
    """
    Perform a k-fold cross-validation on a specific dataset
    given a specific classifier
    
    
    Parameters
    -------------
    - dataset_idx : int
        Dataset index to be called in the data loader
        
    - clf : object
        Classifier object with methods:
        . fit
        . predict
        . score
        
    - k : int (optional)
        Number of folds
        Default: 5
        
    - embeddings_path : str (optional)
        pre-computed embeddings filename
    
    - embeddings : np.ndarray (optional)
        Computed embeddings available in memory
        
    
    Returns
    -----------
    - results : dictionary
        Summary of the results
        Note: the results can be display using
        a pandas.DataFrame such as in:
        ``pd.DataFrame(results)``
    """
    
    # Setup
    scores_val = list()
    scores_train = list()
    
    # Load data
    X_train, Y_train, X_test = load_data(dataset_idx, data_dir=DATA_DIR, files_dict=FILES)
    
    if embeddings_path is not None:
        X_train = np.load(embeddings_path)
        
    if embeddings is not None:
        X_train = embeddings
    
    n = len(X_train)
    assert n == len(Y_train)
    
    # Divise the samples
    bounds = [(i * (n // k), (i+1) * (n // k))
              for i in range(k)]


    # Loop through the divided samples
    for bound in bounds:
        lower, upper = bound
        # Create index array for validation set
        idx = np.arange(lower, upper)
        not_idx = [i for i in range(n) if i not in idx]

        # Populate current train and val sets
        _X_val = X_train[idx]
        _Y_val = Y_train[idx]
        _X_train = X_train[not_idx]
        _Y_train = Y_train[not_idx]

        # Sanity checks
        assert len(_X_train) == len(_Y_train)
        assert len(_X_val) == len(_Y_val)
        assert len(_X_train) == n - len(X_train) // k

        # Fit the classifier on the current training set
        clf.fit(_X_train, _Y_train)
        # Compute the score
        y_pred_train = clf.predict(_X_train)
        y_pred_val = clf.predict(_X_val)
        score_train = clf.score(y_pred_train, _Y_train)
        score_val = clf.score(y_pred_val, _Y_val)

        scores_val.append(score_val)
        scores_train.append(score_train)


   
    # Format the results in a dictionary
    # Compute the score average and standard deviation
    results = {"train_scores": scores_train,
               "val_scores": scores_val,
               "train_avg": np.mean(scores_train),
               "val_avg": np.mean(scores_val),
               "train_std": np.std(scores_train),
               "val_std": np.std(scores_val)}
    
    return results

In [68]:
clf = SVM(_lambda=λ, kernel=kernel)

cross_validation(0, clf)

{'train_scores': [1.0, 1.0, 1.0, 1.0, 1.0],
 'val_scores': [0.5625, 0.5675, 0.575, 0.6, 0.575],
 'train_avg': 1.0,
 'val_avg': 0.576,
 'train_std': 0.0,
 'val_std': 0.012903487900563932}

### Example of Grid Search using Cross Validation on SVM

In [73]:
# Setup
γ = 350
λ = 1e-5
gamma_list = np.linspace(50, γ, 50, endpoint = True)
lambda_list = np.linspace(5e-7, λ, 5, endpoint = True)

settings = list(product(gamma_list,lambda_list))

best_score = {i: 0 for i in range(3)}
best_lambda = {i: 0 for i in range(3)}
best_gamma = {i: 0 for i in range(3)}

for k, tup in enumerate(settings):
    
    γ, λ = tup
    kernel = GaussianKernel(γ)
    
    len_files = len(FILES)
    #clf = SVM(_lambda=λ, kernel=kernel)
    clf = SPR(kernel=kernel)
    
    for i in range(len_files):
        # cross validation (default: k=5)
        results = cross_validation(i, clf, data_dir=DATA_DIR, files_dict=FILES)
        score_train = results["train_avg"]
        score_val = results["val_avg"]
        print(f"Accuracy on train set / val set {i} : {round(score_train, 3)} / {round(score_val, 3)} (λ: {λ},γ: {γ})")
        
        if score_val > best_score[i]:
            best_score[i] = score_val
            best_lambda[i] = λ
            best_gamma[i] = γ
        
    print('\n')

Accuracy on train set / val set 0 : 1.0 / 0.55 (λ: 5e-07,γ: 50.0)
Accuracy on train set / val set 1 : 1.0 / 0.68 (λ: 5e-07,γ: 50.0)
Accuracy on train set / val set 2 : 1.0 / 0.615 (λ: 5e-07,γ: 50.0)


Accuracy on train set / val set 0 : 1.0 / 0.55 (λ: 2.875e-06,γ: 50.0)
Accuracy on train set / val set 1 : 1.0 / 0.68 (λ: 2.875e-06,γ: 50.0)
Accuracy on train set / val set 2 : 1.0 / 0.615 (λ: 2.875e-06,γ: 50.0)


Accuracy on train set / val set 0 : 1.0 / 0.551 (λ: 5.2500000000000006e-06,γ: 50.0)
Accuracy on train set / val set 1 : 1.0 / 0.68 (λ: 5.2500000000000006e-06,γ: 50.0)
Accuracy on train set / val set 2 : 1.0 / 0.615 (λ: 5.2500000000000006e-06,γ: 50.0)


Accuracy on train set / val set 0 : 1.0 / 0.551 (λ: 7.625000000000001e-06,γ: 50.0)


KeyboardInterrupt: 

### Example of Grid Search using Cross Validation on SPR

In [4]:
# Setup
γ = 300
gamma_list = np.linspace(150, γ, 20, endpoint = True)

#gamma_list = [284.7]

#settings = list(product(gamma_list,lambda_list))

best_score = {i: 0 for i in range(3)}
best_gamma = {i: 0 for i in range(3)}

len_files = len(FILES)

#for k, tup in enumerate(settings):
for γ in gamma_list:

    kernel = GaussianKernel(γ)
    
    clf = SPR(kernel=kernel)
    
    for i in range(len_files):
        # cross validation (default: k=5)
        results = cross_validation(i, clf, data_dir=DATA_DIR, files_dict=FILES)
        #print(results)
        score_train = results["train_avg"]
        score_val = results["val_avg"]
        print(f"Accuracy on train set / val set {i} : {round(score_train, 3)} / {round(score_val, 3)} (γ: {γ})")
        
        if score_val > best_score[i]:
            best_score[i] = score_val
            #best_lambda[i] = λ
            best_gamma[i] = γ
        
    print('\n')
    
print(f"Best score: {best_score}")
print(f"Best gamma: {best_gamma}")

Accuracy on train set / val set 0 : 0.731 / 0.554 (γ: 150.0)
Accuracy on train set / val set 1 : 0.875 / 0.647 (γ: 150.0)
Accuracy on train set / val set 2 : 0.768 / 0.603 (γ: 150.0)


Accuracy on train set / val set 0 : 0.742 / 0.55 (γ: 157.89473684210526)
Accuracy on train set / val set 1 : 0.892 / 0.648 (γ: 157.89473684210526)


KeyboardInterrupt: 

### Zero-padding implementation

In [4]:
ENCODING = {'A': [1.,0.,0.,0.],
            'C': [0.,1.,0.,0.],
            'G': [0.,0.,1.,0.],
            'T': [0.,0.,0.,1.],
            'Z': [0.,0.,0.,0.]} # used in zero-padding

def P(i, seq, k, zero_padding=True):
    """
    Compute the a k_mers at a given position in a nucleotides sequence
    
    Parameters
    -----------
    - i : int
        Position in the sequence
        
    - k : int
        Size of k-mer to be returned
        
    - seq : str
        Sequence of nucleotides
        
    - zero_padding : boolean (optional)
        Whether to use zero-padding on the sequence edges
        Default: True
        
    Returns
    -----------
    - L : numpy.array
        One-hot encoding of the string sequence
        
    - not_in : boolean
        Whether the k-mer was computed on the sequence edges
        Always set to False when using zero-padding
    """
    # Setup
    not_in = True
    if zero_padding:
        not_in = False
    
    # lower edge
    if i-(k+1)//2 + 1 < 0:
        # Use heading zero padding here
        n_zeros = abs(i - (k+1) // 2 + 1)
        k_mer_i = 'Z'*n_zeros + seq[:  i + (k+2)//2]
    # upper edge
    elif i + (k+2)//2 > len(seq):
        # Use trailing zero padding here
        n_zeros = i + (k+2) // 2 - len(seq)
        k_mer_i = seq[i - (k+1)//2 + 1:] + 'Z'*n_zeros
    # in the middle
    else:
        k_mer_i = seq[i-(k+1)//2 + 1 :  i + (k+2)//2]
        not_in = False
        
    # concatenate one hot encoding
    L = []
    for c in k_mer_i:
        L += ENCODING[c]
    
    # Sanity check
    assert len(L) == 4 * k
    
    # Convert to array and return
    return np.array(L), not_in

## Investigate ``sklearn`` one-hot encoding implementation

**Bottom-line**: Use *sparse matrix* and/or *Compressed Sparse Row* format

## CKN implementation

**Pseudo-code**: (Unsupervised variant)

0. Input: sequences $(x_i)_{i=\{1, \ldots, n\}}$, labels $(y_i)_{i=\{1, \ldots, n \}}$, $k$ (window-size), $p$ (centroids number), $iter$ (KMeans iteration number), $|A|$ (vocabulary-size: here 4 nucleotides)
1. Process the sequences $x_i$ to obtain $k$-mers.
2. Perform K-Means to obtain $p$ centroids in $\mathbb{R}^{|A|k}$ using $iter$ iterations
3. Compute $K_{ZZ}^{-1/2}$ the (pseudo-)inverse square root Gram matrix defined by $[K_0(z_i, z_j)]_{(i, j) \in \{1, \ldots, p\}^2}$
4. Compute for each training sequence $x$ its mapping $\phi(x) = \frac{1}{m} \sum_{i=1}^m \phi_0(P_i(x)) = \frac{1}{m} \sum_{i=1}^m K_{ZZ}^{-1/2} K_Z(P_i(x))$ where $K_Z(P_i(x)) = [K_0(z_1, P_i(x)), \ldots, K_0(z_p, P_i(x)]^T$
5. Solve $$w^* = \arg \min_{w \in \mathbb{R}^p} \frac{1}{n} \sum_{i=1}^n L(y_i, \langle w, \phi(x_i) \rangle_{\mathbb{R}^p}) + \lambda \parallel w \parallel^2$$
Note that when $L$ is the squared loss, the optimization problems boils down to a Least Square Problem with a $l_2$ regularization, i.e. a Ridge regression problem.
6. Compute for each testing sequence $x'$ its mapping $\phi(x') = \frac{1}{m} \sum_{i=1}^m K_{ZZ}^{-1/2} K_Z(P_i(x'))$
7. Use the $\text{sign}(\langle w^*, \phi(x')\rangle)$ criterion to assign a label to each testing sequence $x'$.

Note: Since the number of k-mers can grow large, it is advisable to use a ``MiniBatch`` K-Means variant. In addition, using the ``kmeans_++`` initialization may be favorable to obtain better results. 

Note: The paper authors mention *extracting a large number of k-mers* which is still quite subjective. We have at most $|A|^k$ unique $k$-mers but the number of duplicates is key to perform ``K-Means`` clustering. 

Note: Granted we use ``zero-padding``, since for each dataset we have 2000 sequences that are all 101 characters long, we can extract at most 202000 $k$-mers per dataset which is huge. However this is still less than $|A|^k$ when $k$ is strictly larger than 8. Since the authors recommend using $k=12$, we have less $k$-mers from our training data than the number of unique $k$-mers that are possible.

## TODOs

1. Process sequences to obtain $k$-mers.
2. Process sequences to obtain a ``sparse`` $k$-mers matrix.
3. Use ``KMeans`` on sequences (sparse VS not sparse)
4. Use ``MiniBatchKMeans`` on sequences (sparse VS not sparse)
5. Compute $K_{ZZ}^{-1/2}$
6. Compute the training sequences mapping
7. Rewrite the optimization problem into a Ridge Regression problem.
8. Compute the testing sequences mapping
9. Apply the ``sign`` criterion to assign a label to each testing sequence.

In [5]:
import sys

In [6]:
%%time
# Process sequences to obtain k-mers
#k = 12
# TODO: Try k=10
k = 10

# Load data
X_train, Y_train, X_test = load_data(0, data_dir=DATA_DIR, files_dict=FILES, mat=False)

n_seqs = len(X_train)
print(f"Number of sequences in training data: {n_seqs}")
seq_length = len(X_train[0])
print(f"Sequence length: {seq_length}")

# Populate the kmers list
kmers = list()
for seq in X_train:
    for i in range(len(seq)):
        kmers.append(P(i, seq, k)[0])

print(f"Number of k-mers processed: {len(kmers)}")
print(f"Memory size of the k-mers: {sys.getsizeof(kmers)} bytes")

Number of sequences in training data: 2000
Sequence length: 101
Number of k-mers processed: 202000
Memory size of the k-mers: 1671792 bytes
CPU times: user 696 ms, sys: 28.9 ms, total: 725 ms
Wall time: 743 ms


In [7]:
from scipy.sparse import lil_matrix

In [8]:
# Convert k-mers to a sparse matrix to be used in MiniBatchKmeans
sparse_matrix = lil_matrix(kmers)
print(f"Memory size of sparse k-mers: {sys.getsizeof(sparse_matrix)} bytes")

Memory size of sparse k-mers: 56 bytes


In [9]:
%%time

# Use MiniBatchKMeans to compute the anchor points 
from sklearn.cluster import MiniBatchKMeans

#n_clusters = 4096
# TODO: try n_clusters = 128
n_clusters = 128

# fit on the whole data
kmeans = MiniBatchKMeans(n_clusters=n_clusters,
        random_state=0,
        batch_size=10000,
        max_iter=5).fit(sparse_matrix)

# Retrieve centroids
anchors = kmeans.cluster_centers_
print(anchors.shape)

(128, 40)
CPU times: user 5.9 s, sys: 39 ms, total: 5.94 s
Wall time: 6.07 s


In [10]:
# Save centroids
np.save("anchors_128_set0.npy", anchors)
# Load centroids
anchors = np.load("anchors_128_set0.npy")
print(anchors.shape)

(128, 40)


In [11]:
# Normalize anchors
anchors_norm = np.linalg.norm(anchors, axis=1).reshape(-1, 1)
normalized_anchors = anchors / anchors_norm
normalized_anchors.shape

(128, 40)

In [12]:
%%time
# Compute K_ZZ in an efficient way
σ = 0.3

K_ZZ = (anchors_norm * anchors_norm.T) * np.power(np.exp(1 / σ**2), normalized_anchors.dot(normalized_anchors.T) - 1)
K_ZZ.shape

CPU times: user 14.3 ms, sys: 16.6 ms, total: 31 ms
Wall time: 21.9 ms


In [13]:
%%time
# Compute K_ZZ^{-1/2} (i.e. pseudo-inverse square root)
K_ZZ_inv_sqrt = scipy.linalg.sqrtm(np.linalg.inv(K_ZZ))

CPU times: user 99.5 ms, sys: 191 ms, total: 291 ms
Wall time: 96.7 ms


**Comment:**

* I checked that using the reverse order in computation (i.e. compute the square root before performing the inversion) yields the same results: ``np.all(K_ZZ_inv_sqrt - K_ZZ_inv_sqrt_2 < 1e-8) >>> True``

In [14]:
# TODO: Compute the training points mapping efficiently

train_kmer = list()
for seq in X_train[:1]:
    for i in range(len(seq)):
        train_kmer.append(P(i, seq, k)[0])
train_kmer[:5]

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        1., 0., 0., 0., 1., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        1., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 1., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 0., 0.])]

In [81]:
K_Z = np.zeros(len(anchors))
print(K_Z.shape)
for l in range(len(anchors)):
    K_Z[l]  = np.linalg.norm(anchors[l]) * np.sqrt(k) * np.power(np.exp(1 / σ**2), normalized_anchors[l].dot(train_kmer[0] / np.sqrt(k)) - 1)
K_Z

(4096,)


array([0.00177207, 0.00136875, 0.00700164, ..., 0.00341937, 0.00153039,
       0.01290227])

In [18]:
# For a single k-mer, we get
K_Z_2 = (anchors_norm.squeeze() * np.sqrt(k)) * np.power(np.exp(1 / σ**2), normalized_anchors.dot(train_kmer[0] / np.sqrt(k))  - 1)
K_Z_2.shape

(128,)

In [19]:
K_ZZ_inv_sqrt.dot(K_Z_2).shape

(128,)

In [20]:
K_Z_2 = (anchors_norm.squeeze() * np.sqrt(k)).reshape(-1, 1) * np.power(np.exp(1 / σ**2), normalized_anchors.dot(np.array(train_kmer).T / np.sqrt(k))  - 1)
K_Z_2.shape

(128, 101)

In [21]:
mean_K_Z = np.mean(K_Z_2, axis=1)
mean_K_Z.shape

(128,)

In [22]:
ϕ_x0 = K_ZZ_inv_sqrt.dot(mean_K_Z)
ϕ_x0.shape

(128,)

In [23]:
type(train_kmer)

list

In [24]:
%%time
# Compute the training point mapping

train_embeddings = np.zeros((n_seqs, n_clusters))

for x, seq in enumerate(X_train):
    curr_kmer = list()
    for i in range(len(seq)):
        curr_kmer.append(P(i, seq, k)[0])
    curr_kmer = np.array(curr_kmer)
    K_Z = ((anchors_norm.squeeze() * np.sqrt(k)).reshape(-1, 1)
           * np.power(np.exp(1 / σ**2), 
                      normalized_anchors.dot(curr_kmer.T / np.sqrt(k)) - 1))
    mean_K_Z = np.mean(K_Z, axis=1)
    train_embeddings[x] = K_ZZ_inv_sqrt.dot(mean_K_Z)

CPU times: user 6.16 s, sys: 6.66 s, total: 12.8 s
Wall time: 3.28 s


In [25]:
train_embeddings.shape

(2000, 128)

In [26]:
# Save train embeddings
np.save("train_embeddings_128_set0.npy", train_embeddings)
# Load train embeddings
train_embeddings = np.load("train_embeddings_128_set0.npy")

In [27]:
train_embeddings.shape

(2000, 128)

## Use ``scikit-learn`` logistic regression

In [28]:
# Use ``scikit-learn`` logistic regression and play around with it

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, solver="sag", C=100).fit(train_embeddings, Y_train)

clf.score(train_embeddings, Y_train)

/home/inzouzouwetrust/miniconda3/envs/km/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.613

In [42]:
clf = LogisticRegression(random_state=0, solver="lbfgs", C=2500, max_iter=2000, tol=1e-6)
#clf = GaussianKernel(γ)


k_split = 5

# Setup
scores_val = list()
scores_train = list()

# Load data
X_train, Y_train, X_test = load_data(0, data_dir=DATA_DIR, files_dict=FILES)
X_train = train_embeddings

n = len(X_train)
assert n == len(Y_train)

# Divise the samples
bounds = [(i * (n // k_split), (i+1) * (n // k_split))
          for i in range(k_split)]


# Loop through the divided samples
for bound in bounds:
    lower, upper = bound
    # Create index array for validation set
    idx = np.arange(lower, upper)
    not_idx = [i for i in range(n) if i not in idx]

    # Populate current train and val sets
    _X_val = X_train[idx]
    _Y_val = Y_train[idx]
    _X_train = X_train[not_idx]
    _Y_train = Y_train[not_idx]

    # Sanity checks
    assert len(_X_train) == len(_Y_train)
    assert len(_X_val) == len(_Y_val)
    assert len(_X_train) == n - len(X_train) // k_split

    # Fit the classifier on the current training set
    clf.fit(_X_train, _Y_train)
    # Compute the score
    y_pred_train = clf.predict(_X_train)
    y_pred_val = clf.predict(_X_val)
    #score_train = clf.score(y_pred_train, _Y_train)
    score_train = np.sum([_Y_train == y_pred_train]) / len(_Y_train)
    #score_val = clf.score(y_pred_val, _Y_val)
    score_val = np.sum([_Y_val == y_pred_val]) / len(_Y_val) 
    
    scores_val.append(score_val)
    scores_train.append(score_train)



# Format the results in a dictionary
# Compute the score average and standard deviation
results = {"train_scores": scores_train,
           "val_scores": scores_val,
           "train_avg": np.mean(scores_train),
           "val_avg": np.mean(scores_val),
           "train_std": np.std(scores_train),
           "val_std": np.std(scores_val)}
results

{'train_scores': [0.648125, 0.663125, 0.661875, 0.655625, 0.6475],
 'val_scores': [0.5925, 0.5575, 0.585, 0.5725, 0.6],
 'train_avg': 0.6552499999999999,
 'val_avg': 0.5815,
 'train_std': 0.0065859699361597536,
 'val_std': 0.01504991694329241}

## Try SVM on new embeddings

In [40]:
# Setup
γ = 200
λ = 1e-5
gamma_list = np.linspace(1, γ, 5, endpoint = True)
lambda_list = np.linspace(1e-7, λ, 3, endpoint = True)

settings = list(product(gamma_list,lambda_list))

best_score = {i: 0 for i in range(3)}
best_lambda = {i: 0 for i in range(3)}
best_gamma = {i: 0 for i in range(3)}

for k, tup in enumerate(settings):
    
    γ, λ = tup
    kernel = GaussianKernel(γ)
    
    len_files = len(FILES)
    clf = SVM(_lambda=λ, kernel=kernel)
    
    #for i in range(len_files):
    for i in range(1):
        # cross validation (default: k=5)
        results = cross_validation(i, clf, embeddings="train_embeddings_128_set0.npy")
        score_train = results["train_avg"]
        score_val = results["val_avg"]
        print(f"Accuracy on train set / val set {i} : {round(score_train, 3)} / {round(score_val, 3)} (λ: {λ},γ: {γ})")
        
        if score_val > best_score[i]:
            best_score[i] = score_val
            best_lambda[i] = λ
            best_gamma[i] = γ
        
    print('\n')

Accuracy on train set / val set 0 : 0.508 / 0.479 (λ: 1e-07,γ: 1.0)


Accuracy on train set / val set 0 : 0.531 / 0.542 (λ: 5.05e-06,γ: 1.0)


Accuracy on train set / val set 0 : 0.567 / 0.542 (λ: 1e-05,γ: 1.0)


Accuracy on train set / val set 0 : 0.957 / 0.576 (λ: 1e-07,γ: 50.75)


Accuracy on train set / val set 0 : 0.726 / 0.598 (λ: 5.05e-06,γ: 50.75)


Accuracy on train set / val set 0 : 0.701 / 0.601 (λ: 1e-05,γ: 50.75)


Accuracy on train set / val set 0 : 1.0 / 0.571 (λ: 1e-07,γ: 100.5)


Accuracy on train set / val set 0 : 0.794 / 0.605 (λ: 5.05e-06,γ: 100.5)


Accuracy on train set / val set 0 : 0.757 / 0.604 (λ: 1e-05,γ: 100.5)


Accuracy on train set / val set 0 : 1.0 / 0.574 (λ: 1e-07,γ: 150.25)


Accuracy on train set / val set 0 : 0.849 / 0.597 (λ: 5.05e-06,γ: 150.25)


Accuracy on train set / val set 0 : 0.801 / 0.604 (λ: 1e-05,γ: 150.25)


Accuracy on train set / val set 0 : 1.0 / 0.572 (λ: 1e-07,γ: 200.0)


Accuracy on train set / val set 0 : 0.89 / 0.59 (λ: 5.05e-06,